In [1]:
import logging, os
logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import geopandas as gpd
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import datetime
from pathlib import Path

import plotly.express as px

import tensorflow as tf

from tensorflow.keras.layers import *
from tensorflow.keras.initializers import *

from sklearn.model_selection import train_test_split


import keras_tuner as kt

from tqdm.notebook import tqdm

from importlib import reload
import sentinel_utils
import plot_utils
from data_generator import DataGenerator

In [2]:
class BuildHyperModel(kt.HyperModel):
    def __init__(self, output_shape, output_bias):
        super().__init__()
        self.output_shape = output_shape
        self.output_bias = output_bias

    def get_metrics(self):
        prc = tf.keras.metrics.AUC(name='prc', curve='PR')
        f1_score = tf.keras.metrics.FBetaScore(
            average='weighted', beta=1.0, threshold=0.5, name=f'weightedf1score'
        )
        f2_score = tf.keras.metrics.FBetaScore(
            average='weighted', beta=2.0, threshold=0.5, name=f'weightedf2score'
        )
        metrics = [
            'accuracy', 'recall', 'precision', 'auc', prc, f1_score, f2_score
        ] 
        return metrics
        
    def build(self, hp):
        sentinel_10m_input = Input((100, 100, 2))
        sentinel_20m_input = Input((50, 50, 2))

        x = concatenate([sentinel_10m_input, UpSampling2D(2)(sentinel_20m_input)])

        pooling = {'max': MaxPooling2D, 'average': AveragePooling2D}
        pooling_choice = hp.Choice('pooling', list(pooling.keys()))
        
        filter_power = hp.Int('filters_power', min_value=4, max_value=9, step=1)
        kernel_size = hp.Choice('kernel_size', [3, 5])
        for filters_scale in [2**x for x in range(3, filter_power+1)]:
            x = Conv2D(
                filters=filters_scale,
                kernel_size=kernel_size,
                padding='same',
                activation='relu',
            )(x)
            x = pooling[pooling_choice](pool_size=2, strides=2, padding='same')(x)    
            x = BatchNormalization()(x)
        
        x = Flatten()(x)

        units_power = hp.Int(
            'units_power', min_value=4, max_value=10, step=1
        )
        for units_scale in reversed([2**x for x in range(4, filter_power+1)]):
            x = Dense(units_scale, activation='relu')(x)
            x = BatchNormalization()(x)

        x = Dropout(hp.Float('dropout_rate', min_value=0.0, max_value=0.6, step=0.1))(x)

        use_bias = hp.Boolean('output_bias')
        if use_bias:
            output_bias = tf.keras.initializers.Constant(self.output_bias)
        else:
            output_bias = None
            
        outputs = Dense(
            self.output_shape,
            bias_initializer=output_bias,
            activation='sigmoid',   
        )(x)
        m = tf.keras.models.Model(
            inputs=[sentinel_10m_input, sentinel_20m_input], 
            outputs=outputs
        )
        m.compile(
            optimizer='adam',
            loss=hp.Choice('loss', ['binary_crossentropy', 'binary_focal_crossentropy']), 
            metrics=self.get_metrics()
        )
        return m

In [3]:
reload(sentinel_utils)

model_dir = Path('models', 'selected_model')
shards_dir = Path.home().joinpath('sentinel_data', 'shards')

utils = sentinel_utils.SentinelUtils(min_occurrences=20000)

selected_classes = utils.get_processed_labels()

data_summary = utils.get_data_summary(
    shards_dir, selected_classes, overwrite_existing=False
)

params = dict(
    shards_dir=shards_dir,
    selected_classes=selected_classes,
    data_summary=data_summary,
    batch_size=64
)

In [4]:
trials_dir = Path('trials', 'hyperband')
trial_metric = 'val_weightedf2score'

In [5]:
callbacks = [
    tf.keras.callbacks.TensorBoard(trials_dir.joinpath('board')),
    tf.keras.callbacks.EarlyStopping(
        monitor=trial_metric, patience=10, mode='max'
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor=trial_metric, factor=0.5, patience=4, min_lr=1e-7,
    ),
]

In [ ]:
training_ids, test_ids = train_test_split(
    selected_classes.index, test_size=10000, random_state=42
)

training_generator = DataGenerator(training_ids, shuffle=True, **params)
testing_generator = DataGenerator(test_ids, shuffle=False, **params)

hypermodel = BuildHyperModel(
    output_shape=selected_classes.shape[1], 
    output_bias=data_summary['initial_bias']
)

tuner = kt.Hyperband(
    hypermodel,
    objective=kt.Objective(trial_metric, direction='max'),
    directory=trials_dir.parent,
    project_name=trials_dir.name,
    hyperband_iterations=3,
    max_epochs=50,
    overwrite=False,
)
     
tuner.search(
    x=training_generator,
    validation_data=testing_generator,
    class_weight=data_summary['class_weights'],
    callbacks=callbacks,
    verbose=1
)

Trial 65 Complete [00h 13m 15s]
val_weightedf2score: 0.5889411568641663

Best val_weightedf2score So Far: 0.6650234460830688
Total elapsed time: 12h 14m 49s

Search: Running Trial #66

Value             |Best Value So Far |Hyperparameter
max               |average           |pooling
4                 |8                 |filters_power
5                 |3                 |kernel_size
7                 |10                |units_power
0.1               |0                 |dropout_rate
False             |False             |output_bias
binary_focal_cr...|binary_focal_cr...|loss
6                 |17                |tuner/epochs
0                 |6                 |tuner/initial_epoch
2                 |3                 |tuner/bracket
0                 |2                 |tuner/round

Epoch 1/6
3550/3550 ━━━━━━━━━━━━━━━━━━━━ 134s 37ms/step - accuracy: 0.3864 - auc: 0.8106 - loss: 0.1241 - prc: 0.5680 - precision: 0.5750 - recall: 0.5496 - weightedf1score: 0.5289 - weightedf2score: 0.5380 -

In [ ]:
# tuner.get_best_models(num_models=5)[0].fit(
#     x=training_generator,
#     validation_data=testing_generator,
#     class_weight=data_summary['class_weights'],
#     epochs=10,
#     verbose=1
# )

In [ ]:
tuner.results_summary(num_trials=10)